In [1]:
import json
import os
import pandas as pd
import ray

from pprint import pprint 

from env import QdTreeEnv
from qdtree import Workload
from qdtree.schema import ensure_data_schema
from policy import QdTreePolicy

from ray.rllib.env.env_context import EnvContext
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
from ray import air, tune


Parameters

In [2]:
# We use sf = 1 for TPC-H. The paper uses sf = 1000
BASE_PATH = "../data/tpc-h/"
# The sampling rate in the paper is 0.1
SAMPLING_RATE = 0.01
# The min block size in the paper is 100K, we use 100 to scale down according to the sf
MIN_BLOCK_SIZE = 1000 * SAMPLING_RATE

# Load the workload

In [3]:
WORKLOAD_PATH = os.path.join(BASE_PATH, "workload3.json")
with open(WORKLOAD_PATH, "r") as f:
    workload = Workload(json.load(f))

workload._queries

{'1.17.sql': l_shipdate <= 906098400,
 '1.100.sql': l_shipdate <= 906530400,
 '1.16.sql': l_shipdate <= 906789600,
 '1.14.sql': l_shipdate <= 902901600,
 '1.28.sql': l_shipdate <= 903765600,
 '1.29.sql': l_shipdate <= 903074400,
 '1.15.sql': l_shipdate <= 902210400,
 '1.39.sql': l_shipdate <= 906703200,
 '1.11.sql': l_shipdate <= 904975200,
 '1.10.sql': l_shipdate <= 905666400,
 '1.38.sql': l_shipdate <= 902124000,
 '1.12.sql': l_shipdate <= 904284000,
 '1.13.sql': l_shipdate <= 903592800,
 '1.74.sql': l_shipdate <= 903506400,
 '1.60.sql': l_shipdate <= 902642400,
 '1.48.sql': l_shipdate <= 905666400,
 '1.49.sql': l_shipdate <= 904975200,
 '1.61.sql': l_shipdate <= 907221600,
 '1.75.sql': l_shipdate <= 902815200,
 '1.63.sql': l_shipdate <= 905839200,
 '1.77.sql': l_shipdate <= 906703200,
 '1.88.sql': l_shipdate <= 904284000,
 '1.89.sql': l_shipdate <= 903592800,
 '1.76.sql': l_shipdate <= 902124000,
 '1.62.sql': l_shipdate <= 906530400,
 '1.66.sql': l_shipdate <= 903765600,
 '1.72.sql'

# Load and sample data

In [4]:
DATA_PATH = os.path.join(BASE_PATH, "sf10_2/denormalized.parquet")
all_data = pd.read_parquet(DATA_PATH)
all_data.columns = all_data.columns.str.lower()
all_data = ensure_data_schema(all_data, workload.schema)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5998605 entries, 306071 to 269806
Data columns (total 61 columns):
 #   Column           Dtype  
---  ------           -----  
 0   l_orderkey       int64  
 1   l_partkey        int64  
 2   l_suppkey        int64  
 3   l_linenumber     object 
 4   l_quantity       float64
 5   l_extendedprice  float64
 6   l_discount       float64
 7   l_tax            float64
 8   l_returnflag     object 
 9   l_linestatus     object 
 10  l_shipdate       int64  
 11  l_commitdate     int64  
 12  l_receiptdate    int64  
 13  l_shipinstruct   object 
 14  l_shipmode       object 
 15  l_comment        object 
 16  o_orderkey       object 
 17  o_custkey        int64  
 18  o_orderstatus    object 
 19  o_totalprice     float64
 20  o_orderdate      int64  
 21  o_orderpriority  object 
 22  o_clerk          object 
 23  o_shippriority   object 
 24  o_comment        object 
 25  c_custkey        object 
 26  c_name           object 
 27  c_addres

In [ ]:
all_data.columns

In [5]:
data = all_data.sample(frac=SAMPLING_RATE)
len(data)

59986

In [6]:
del all_data

# Set up the environment

In [7]:
ray.init(local_mode=False, ignore_reinit_error=True,num_cpus=8,_plasma_directory="/tmp",_system_config={
        "object_spilling_config": json.dumps(
            {"type": "filesystem", "params": {"directory_path": "/tmp/spill"}},
        )
    })

2024-05-20 19:06:14,738	INFO services.py:1805 -- object_store_memory is not verified when plasma_directory is set.
2024-05-20 19:06:14,864	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.10.14
Ray version:,2.3.1


(raylet) /opt/miniconda3/envs/qdtree/lib/python3.10/site-packages/ray/dashboard/modules/reporter/reporter_agent.py:56: UserWarning: `gpustat` package is not installed. GPU monitoring is not available. To have full functionality of the dashboard please install `pip install ray[default]`.)
(raylet)   warnings.warn(
(RolloutWorker pid=42648) 2024-05-20 19:06:33,156	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=42648) 2024-05-20 19:06:33,156	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
(PPO pid=42652) 2024-05-20 19:06:40,521	WARNING algorithm_config.py:596 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(PPO pid=42652) 2024-

In [8]:
env_config = {
    "workload": workload,
    "data": data,
    "min_leaf_size": MIN_BLOCK_SIZE,
}

config = (
    PPOConfig()
    .environment(
        QdTreeEnv,
        env_config=env_config,
    )
    .framework("torch")
    .rollouts(num_rollout_workers=2, batch_mode="complete_episodes")
    .training(
        model={
            "fcnet_hiddens": [512, 512],
            "fcnet_activation": "relu",
        }
    )
)

algo = config.build()

stop = {
    "training_iteration": 10,
    "timesteps_total": 40000,
}

2024-05-20 19:06:30,639	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2024-05-20 19:06:33,723	WARNING util.py:67 -- Install gputil for GPU system monitoring.


In [9]:
os.chdir('/Users/umsaka/Documents/DDCPS/qdtree/src')

In [10]:
tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(
        local_dir="/Users/umsaka/Documents/DDCPS/qdtree/src/results2",
        stop=stop,
        checkpoint_config=air.CheckpointConfig(checkpoint_at_end=True),
    ),
)

result = tuner.fit()

2024-05-20 19:06:38,777	WARNING util.py:244 -- The `start_trial` operation took 0.686 s, which may be a performance bottleneck.


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_QdTreeEnv_60042_00000,40078,"{'ObsPreprocessorConnector_ms': 0.014281794021689827, 'StateBufferConnector_ms': 0.005456398093635267, 'ViewRequirementAgentConnector_ms': 0.1020567013266308}","{'num_env_steps_sampled': 40078, 'num_env_steps_trained': 40078, 'num_agent_steps_sampled': 40078, 'num_agent_steps_trained': 40078}",{},2024-05-20_19-13-00,True,10.9454,{},0.518378,0.28355,0.0708897,366,3860,acf834784a114d8fab7254ae88587325,Mete-New.local,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 1.7771310039104955, 'cur_kl_coeff': 1.5187500000000005, 'cur_lr': 5.0000000000000016e-05, 'total_loss': -0.08688468707184638, 'policy_loss': -0.12351649214303301, 'vf_loss': 0.01120877194779134, 'vf_explained_var': 0.00956154292629611, 'kl': 0.016739446088590113, 'entropy': 5.5971402101619265, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 8835.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 40078, 'num_env_steps_trained': 40078, 'num_agent_steps_sampled': 40078, 'num_agent_steps_trained': 40078}",10,127.0.0.1,40078,40078,40078,4006,40078,4006,0,2,0,0,4006,"{'cpu_util_percent': 40.33157894736841, 'ram_util_percent': 81.86315789473684}",42652,{},{},{},"{'mean_raw_obs_processing_ms': 1.315387024199628, 'mean_inference_ms': 1.4198331396354218, 'mean_action_processing_ms': 0.0639462336037297, 'mean_env_wait_ms': 9.940865344713924, 'mean_env_render_ms': 0.0}",370.22,39.3798,370.22,"{'training_iteration_time_ms': 37013.549, 'load_time_ms': 5.777, 'load_throughput': 693753.779, 'learn_time_ms': 11138.575, 'learn_throughput': 359.813, 'synch_weights_time_ms': 4.393}",1716253980,0,40078,10,60042_00000,3.73503


2024-05-20 19:13:04,896	INFO tune.py:798 -- Total run time: 386.99 seconds (383.84 seconds for the tuning loop).


In [11]:
agent = config.build()
agent.restore(result.get_best_result().checkpoint) # type: ignore

env = QdTreeEnv(EnvContext(env_config, 0))
done = False
obs, _ = env.reset() 
step = 0
episode_reward = 0
while not done:
    action = agent.compute_single_action(obs, explore=False)
    obs, reward, done, _, info = env.step(action) # type: ignore
    episode_reward += reward
    # print(step, action, obs, reward, done, info)
    step += 1

pprint(env.qd_tree.blocks)
print(episode_reward)


2024-05-20 19:14:06,889	WARNING util.py:67 -- Install gputil for GPU system monitoring.
2024-05-20 19:14:07,084	WARNING algorithm_config.py:596 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2024-05-20 19:14:07,489	INFO trainable.py:791 -- Restored on 127.0.0.1 from checkpoint: /Users/umsaka/Documents/DDCPS/qdtree/src/results2/PPO/PPO_QdTreeEnv_60042_00000_0_2024-05-20_19-06-38/checkpoint_000010
2024-05-20 19:14:07,489	INFO trainable.py:800 -- Current state after restoring: {'_iteration': 10, '_timesteps_total': None, '_time_total': 370.22036242485046, '_episodes_total': 3860}


[{'c_acctbal': (-inf, inf),
  'c_address': (-inf, inf),
  'c_comment': (-inf, inf),
  'c_custkey': (-inf, inf),
  'c_mktsegment': (-inf, inf),
  'c_name': (-inf, inf),
  'c_nationkey': (-inf, inf),
  'c_phone': (-inf, inf),
  'l_comment': (-inf, inf),
  'l_commitdate': (-inf, inf),
  'l_discount': (-inf, inf),
  'l_extendedprice': (-inf, inf),
  'l_linenumber': (-inf, inf),
  'l_linestatus': (-inf, inf),
  'l_orderkey': (-inf, inf),
  'l_partkey': (-inf, inf),
  'l_quantity': (-inf, inf),
  'l_receiptdate': (-inf, inf),
  'l_returnflag': (-inf, inf),
  'l_shipdate': [786265200, inf),
  'l_shipinstruct': (-inf, inf),
  'l_shipmode': (-inf, inf),
  'l_suppkey': (-inf, inf),
  'l_tax': (-inf, inf),
  'n_comment': (-inf, inf),
  'n_name': (-inf, inf),
  'n_nationkey': (-inf, inf),
  'n_regionkey': (-inf, inf),
  'o_clerk': (-inf, inf),
  'o_comment': (-inf, inf),
  'o_custkey': (-inf, inf),
  'o_orderdate': [823158000, inf),
  'o_orderkey': (-inf, inf),
  'o_orderpriority': (-inf, inf),
  

In [35]:
print(type(env.qd_tree.blocks))
env.qd_tree.blocks

<class 'list'>


[{'l_orderkey': (-inf, inf),
  'l_partkey': (-inf, inf),
  'l_suppkey': (-inf, inf),
  'l_linenumber': (-inf, inf),
  'l_quantity': (-inf, inf),
  'l_extendedprice': (-inf, inf),
  'l_discount': (-inf, inf),
  'l_tax': (-inf, inf),
  'l_returnflag': (-inf, inf),
  'l_linestatus': (-inf, inf),
  'l_shipdate': [786265200, inf),
  'l_commitdate': (-inf, inf),
  'l_receiptdate': (-inf, inf),
  'l_shipinstruct': (-inf, inf),
  'l_shipmode': (-inf, inf),
  'l_comment': (-inf, inf),
  'o_orderkey': (-inf, inf),
  'o_custkey': (-inf, inf),
  'o_orderstatus': (-inf, inf),
  'o_totalprice': (-inf, inf),
  'o_orderdate': [823158000, inf),
  'o_orderpriority': (-inf, inf),
  'o_clerk': (-inf, inf),
  'o_shippriority': (-inf, inf),
  'o_comment': (-inf, inf),
  'c_custkey': (-inf, inf),
  'c_name': (-inf, inf),
  'c_address': (-inf, inf),
  'c_nationkey': (-inf, inf),
  'c_phone': (-inf, inf),
  'c_acctbal': (-inf, inf),
  'c_mktsegment': (-inf, inf),
  'c_comment': (-inf, inf),
  'n_nationkey': (-

In [41]:
(env.qd_tree.blocks[0])

{'l_orderkey': (-inf, inf),
 'l_partkey': (-inf, inf),
 'l_suppkey': (-inf, inf),
 'l_linenumber': (-inf, inf),
 'l_quantity': (-inf, inf),
 'l_extendedprice': (-inf, inf),
 'l_discount': (-inf, inf),
 'l_tax': (-inf, inf),
 'l_returnflag': (-inf, inf),
 'l_linestatus': (-inf, inf),
 'l_shipdate': [786265200, inf),
 'l_commitdate': (-inf, inf),
 'l_receiptdate': (-inf, inf),
 'l_shipinstruct': (-inf, inf),
 'l_shipmode': (-inf, inf),
 'l_comment': (-inf, inf),
 'o_orderkey': (-inf, inf),
 'o_custkey': (-inf, inf),
 'o_orderstatus': (-inf, inf),
 'o_totalprice': (-inf, inf),
 'o_orderdate': [823158000, inf),
 'o_orderpriority': (-inf, inf),
 'o_clerk': (-inf, inf),
 'o_shippriority': (-inf, inf),
 'o_comment': (-inf, inf),
 'c_custkey': (-inf, inf),
 'c_name': (-inf, inf),
 'c_address': (-inf, inf),
 'c_nationkey': (-inf, inf),
 'c_phone': (-inf, inf),
 'c_acctbal': (-inf, inf),
 'c_mktsegment': (-inf, inf),
 'c_comment': (-inf, inf),
 'n_nationkey': (-inf, inf),
 'n_name': (-inf, inf),

In [43]:
# with open("qd_tree.json", "w") as f:
#     json.dump(str(env.qd_tree), f)
json.dumps(env.qd_tree.blocks)
# for dicty in env.qd_tree.blocks:
#     with open('blocks.json', 'a') as f:
#         json.dump(dicty, f,indent=4)

TypeError: Object of type RangeWithDict is not JSON serializable

In [53]:
for i in env.qd_tree.blocks:
    for key in i:
        i[key] = [i[key].left,i[key].right]


In [54]:
with open('blocks.json', 'w') as f:
    json.dump(env.qd_tree.blocks, f,indent=4)

In [56]:
with open('blocks.json', 'r') as f:
    blocks = json.load(f)


In [57]:
print(blocks)

[{'l_orderkey': [-inf, inf], 'l_partkey': [-inf, inf], 'l_suppkey': [-inf, inf], 'l_linenumber': [-inf, inf], 'l_quantity': [-inf, inf], 'l_extendedprice': [-inf, inf], 'l_discount': [-inf, inf], 'l_tax': [-inf, inf], 'l_returnflag': [-inf, inf], 'l_linestatus': [-inf, inf], 'l_shipdate': [786265200, inf], 'l_commitdate': [-inf, inf], 'l_receiptdate': [-inf, inf], 'l_shipinstruct': [-inf, inf], 'l_shipmode': [-inf, inf], 'l_comment': [-inf, inf], 'o_orderkey': [-inf, inf], 'o_custkey': [-inf, inf], 'o_orderstatus': [-inf, inf], 'o_totalprice': [-inf, inf], 'o_orderdate': [823158000, inf], 'o_orderpriority': [-inf, inf], 'o_clerk': [-inf, inf], 'o_shippriority': [-inf, inf], 'o_comment': [-inf, inf], 'c_custkey': [-inf, inf], 'c_name': [-inf, inf], 'c_address': [-inf, inf], 'c_nationkey': [-inf, inf], 'c_phone': [-inf, inf], 'c_acctbal': [-inf, inf], 'c_mktsegment': [-inf, inf], 'c_comment': [-inf, inf], 'n_nationkey': [-inf, inf], 'n_name': [-inf, inf], 'n_regionkey': [-inf, inf], 'n_c